# Video Compression Assignment 3

In this assignment, we will use the block-based encoding approach, where the size of a block is 8x8. Only the Luma component is considered for the following questions.

### Prerequisites

- Programming langauge: Python 3.10+ (IPython)
- Framework: Jupyter

Install dependencies from PyPI:

In [10]:
# %pip install \
#     --disable-pip-version-check \
#     --quiet \
#     numpy \
#     Pillow

## Task 1

> Fourier Transform

Please apply the Fourier Transform to the luma component of `foreman_qcif_0_rgb.bmp` and demonstrate its magnitudes in a 2-D image, as shown in the example below. Note that you need to shift the origin to the center of the image for the magnitude plot.

<div style="text-align: center;">
    <img alt="Image 1" src="https://raw.githubusercontent.com/AsherJingkongChen/video-compression-assignment-3/main/notebooks/images/1.png" width="50%">
</div>

## Solution of Task 1

## Task 2

> DCT

Please apply DCT to all the 8x8 luma blocks of `foreman_qcif_0_rgb.bmp` and use the quantization matrix below for quantization. After DCT and quantization, please apply inverse quantization and IDCT to decode all the blocks and show the decoded frame.

$$
\begin{bmatrix}
16 & 11 & 10 & 16 & 24 & 40 & 51 & 61 \\
12 & 12 & 14 & 19 & 26 & 58 & 60 & 55 \\
14 & 13 & 16 & 24 & 40 & 57 & 69 & 56 \\
14 & 17 & 22 & 29 & 51 & 87 & 80 & 62 \\
18 & 22 & 37 & 56 & 68 & 109 & 103 & 77 \\
24 & 35 & 55 & 64 & 81 & 104 & 113 & 92 \\
49 & 64 & 78 & 87 & 103 & 121 & 120 & 101 \\
72 & 92 & 95 & 98 & 112 & 100 & 103 & 99 \\
\end{bmatrix}
$$

## Solution of Task 2

### 2-D Discrete Cosine Transform

#### Forward Transform (for `NxN` block)

The formula adapted from the course slides:

$$
F(u, v) = \frac{2}{N}C(u)C(v)\sum_{x=0}^{N-1}\sum_{y=0}^{N-1}f(x, y)\cos\left(\frac{(2x+1)u\pi}{2N}\right)\cos\left(\frac{(2y+1)v\pi}{2N}\right) \\
\text{where } C(t) = \begin{cases} \frac{2}{\sqrt{N}} & \text{if } t = 0 \\ 2 \sqrt{\frac{2}{N}} & \text{otherwise} \end{cases}$$

can be simplified as:

$$
F(u, v) = \frac{8}{N^{2}}C(u)C(v)\sum_{x=0}^{N-1}\sum_{y=0}^{N-1}f(x, y)\cos\left(\frac{(2x+1)u\pi}{2N}\right)\cos\left(\frac{(2y+1)v\pi}{2N}\right) \\
\text{where } C(t) = \begin{cases} 1 & \text{if } t = 0 \\ \sqrt{2} & \text{otherwise} \end{cases}$$

#### Inverse Transform (for `NxN` block)

$$
f(x, y) = \frac{2}{N}\sum_{u=0}^{N-1}\sum_{v=0}^{N-1}C(u)C(v)F(u, v)\cos\left(\frac{(2x+1)u\pi}{2N}\right)\cos\left(\frac{(2y+1)v\pi}{2N}\right) \\
\text{where } C(t) = \begin{cases} \frac{2}{\sqrt{N}} & \text{if } t = 0 \\ 2 \sqrt{\frac{2}{N}} & \text{otherwise} \end{cases}
$$

can be simplified as:

$$
f(x, y) = \frac{8}{N^{2}}\sum_{u=0}^{N-1}\sum_{v=0}^{N-1}C(u)C(v)F(u, v)\cos\left(\frac{(2x+1)u\pi}{2N}\right)\cos\left(\frac{(2y+1)v\pi}{2N}\right) \\
\text{where } C(t) = \begin{cases} 1 & \text{if } t = 0 \\ \sqrt{2} & \text{otherwise} \end{cases}$$

Implement the DCT and IDCT functions:

In [119]:
from numpy.typing import NDArray
from numpy import float64, uint8


def dct_forward(source: NDArray[uint8], block_size: int) -> NDArray[float64]:
    from numpy import zeros_like, asfarray, atleast_2d
    from math import sqrt, cos, pi

    source = asfarray(atleast_2d(source))

    N = block_size
    C = 8 / N / N
    PI_2N = pi / 2 / N
    SR_2 = sqrt(2)

    target = source.copy()
    for i in range(0, target.shape[0] - N + 1, N):
        for j in range(0, target.shape[1] - N + 1, N):
            f = source[i : i + N, j : j + N]
            g = zeros_like(f)
            for u in range(N):
                for v in range(N):
                    a = 0.0
                    for x in range(N):
                        for y in range(N):
                            a += (
                                f[x, y]
                                * cos((2 * x + 1) * u * PI_2N)
                                * cos((2 * y + 1) * v * PI_2N)
                            )
                    g[u, v] = C * (1 if u == 0 else SR_2) * (1 if v == 0 else SR_2) * a
            target[i : i + N, j : j + N] = g
    return target


def dct_inverse(source: NDArray[float64], block_size: int) -> NDArray[uint8]:
    from numpy import zeros_like, asfarray, atleast_2d, uint8
    from math import cos, pi, sqrt

    source = asfarray(atleast_2d(source))

    N = block_size
    C = 8 / N / N
    PI_2N = pi / 2 / N
    SR_2 = sqrt(2)

    target = source.copy()
    for i in range(0, target.shape[0] - N + 1, N):
        for j in range(0, target.shape[1] - N + 1, N):
            g = source[i : i + N, j : j + N]
            f = zeros_like(g)
            for x in range(N):
                for y in range(N):
                    a = 0.0
                    for u in range(N):
                        for v in range(N):
                            a += (
                                (1 if u == 0 else SR_2)
                                * (1 if v == 0 else SR_2)
                                * g[u, v]
                                * cos((2 * x + 1) * u * PI_2N)
                                * cos((2 * y + 1) * v * PI_2N)
                            )
                    f[x, y] = round(C * a)
            target[i : i + N, j : j + N] = f
    target = target.round().astype(uint8)
    return target

Test the DCT and IDCT functions:

In [151]:
from numpy.random import randint
from numpy import set_printoptions

mat = randint(low=0, high=256, size=(9, 8), dtype=uint8)
mat_dct = dct_forward(mat, 8)
mat_idct = dct_inverse(mat_dct, 8)

set_printoptions(suppress=True)
print(mat_dct.round(1))

assert (mat == mat_idct).all()

[[ 961.   -35.3  -31.7 ... -115.     3.1 -105.4]
 [-129.7  155.7   11.  ...   13.3 -100.1  -31.1]
 [  17.8   21.3  -65.1 ...   35.8   92.7   22.8]
 ...
 [  56.   -46.2   40.9 ...   -5.7  -55.6  -24.8]
 [ 141.5  -71.6  -62.1 ...  -99.1  116.2   21.4]
 [ 200.   202.     4.  ...  247.    40.    59. ]]


In [11]:
%%html
<style>
/* Do not render this cell */
@media print {
  .jp-Cell.jp-Notebook-cell:last-child {
    display: none;
  }
}

:root {
  --jp-content-link-color: dodgerblue;
}
@page {
  size: A3 portrait;
  margin: 5mm;
}
@media screen {
    body {
        margin: 5mm;
    }
}
a code {
    color: var(--jp-content-link-color) !important;
}
.jp-RenderedHTMLCommon {
  font-family: Calibri, Verdana, sans-serif !important;
  font-size: 13px !important;
  font-weight: 400 !important;
  line-height: 1.35 !important;
}
code, pre {
    font-family: Monaco, monospace !important;
}
h1 {
    text-align: center !important;
}
h1, h2, h3, h4, h5, h6, strong {
    font-weight: 700 !important;
}
h2 {
  page-break-before: always;
}
pre {
  white-space: pre-wrap;
}
table, td, th, tr, tbody, thead, tfoot {
  page-break-inside: avoid !important;
}
.jp-RenderedHTMLCommon td,
.jp-RenderedHTMLCommon th,
.jp-RenderedHTMLCommon tr {
  border: 1px solid var(--md-grey-500);
}
.jp-RenderedHTMLCommon table {
  margin-left: 2em;
}
.jp-CodeCell {
    margin-bottom: 1.5em;
}
</style>